In [1]:
#import lxml.html
#import requests
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import time
#import math
import re
import json
import plotly.graph_objects as go
import plotly.express as px
#from bs4 import BeautifulSoup as bs4
#from selenium.webdriver import Chrome
#from selenium import webdriver
#from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

import optimize_lineup as ol
import fantasy_utils as fu

In [2]:
team_list = ['Lima Time!', 'Ugly Spuds', 'Harveys Wallbangers', 'Roid Ragers', 'Charmer', 'Wiscompton Wu-Tang', '10 O\'Clock Madness', 'Brewbirds', 'Trouble With The Curve', 'Lil Trump & The Ivanabees', 'Dirty Birds', 'Young Guns'] 
eligibility_url_dict = {"8 Grand Kids":'30', 'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', "Harveys Wallbangers":'4', 'Lil Trump & the Ivanabees':'27', 'Lima Time!':'38', 
                        'Roid Ragers':'44', 'Trouble with the Curve':'1', 'Ugly Spuds':'29', 'Wiscompton Wu-Tang':'42', 'Young Guns':'45'}
stat_list = ['BA', 'HR', 'R', 'RBI', 'SB', 'ERA', 'Sv+Hld', 'SO', 'W', 'WHIP']

In [13]:
def load():
    df = pd.read_csv('data\\period_6_ros_projections_.csv')
    df.loc[df['Player'].isin(['Nico Hoerner', 'Bryson Stott']), 'all_pos'] = "['2B', 'SS', 'MI', 'DH']"
    df.loc[df['Player'].isin(['Isaac Paredes', ]), 'all_pos'] = "['2B', '3B', '1B', 'MI', 'CI']"
    df.loc[df['Player'].isin(['Max Muncy', ]), 'all_pos'] = "['2B', '3B', 'MI', 'CI']"
    return df

In [4]:
df[(df['Start']==1)].groupby('Owner').apply(lambda s: pd.Series({
    'BA':round(s['H'].sum()/s['AB'].sum(),3),
    'HR':s['HR'].sum(),
    'SB':s['SB'].sum(),
    'R':s['R'].sum(),
    'RBI':s['RBI'].sum(),
    'W': s['W'].sum(),
    'SO':s['SO'].sum(),
    'S':s['S'].sum()+s['HD'].sum(),
    'ERA':round(s['ER'].sum()/s['IP'].sum()*9,2),
    'WHIP':round((s['Ha'].sum()+s['BBa'].sum())/s['IP'].sum(),2),
}))

,BA,HR,SB,R,RBI,W,SO,S,ERA,WHIP
Owner,,,,,,,,,,
Brewbirds,0.277,237.0,137.0,914.0,850.0,75.0,1134.0,51.0,3.36,1.15
Charmer,0.276,273.0,116.0,858.0,905.0,73.0,1128.0,53.0,3.33,1.14
Dirty Birds,0.283,226.0,153.0,915.0,846.0,57.0,1045.0,91.0,3.11,1.17
Harvey,0.271,284.0,165.0,949.0,880.0,85.0,1036.0,22.0,3.20,1.14
Lil Trump,0.262,266.0,54.0,851.0,891.0,59.0,1043.0,79.0,3.44,1.12
Lima Time,0.279,254.0,102.0,853.0,859.0,91.0,1286.0,0.0,3.37,1.20
Madness,0.285,239.0,151.0,924.0,880.0,60.0,985.0,96.0,3.36,1.14
Roiders,0.285,270.0,89.0,938.0,921.0,76.0,1080.0,66.0,3.12,1.13
Trouble,0.266,266.0,132.0,894.0,897.0,67.0,1021.0,91.0,3.49,1.18


In [5]:
def optimize_team(tm):
    w = ol.Optimized_Lineups(tm, df[df['Owner']==tm])
    print(tm)
    w._make_pitcher_combos()
    w._make_hitter_combos()
    print(w.pitcher_optimized_z, w.pitcher_optimized_lineup)
    print(w.hitter_optimized_z, w.hitter_optimized_lineup)
    return w

In [59]:
h = ol.Optimized_Lineups('Harvey', df[df['Owner']=='Harvey'])
w = ol.Optimized_Lineups('Wu Tang', df[df['Owner']=='Wu Tang'])
l = ol.Optimized_Lineups('Lima Time', df[df['Owner']=='Lima Time'])
t = ol.Optimized_Lineups('Trouble', df[df['Owner']=='Trouble'])
u = ol.Optimized_Lineups('Ugly Spuds', df[df['Owner']=='Ugly Spuds'])

In [6]:
tm = 'Lima Time'
l = optimize_team(tm)

Lima Time
169632
169632
5.520648398669712 ('Justin Verlander', 'Shane McClanahan', 'Cristian Javier', 'Lucas Giolito', 'Charlie Morton', 'Justin Steele', 'Jordan Montgomery', 'Erik Swanson', 'Jason Adam')
-4.403569986971904 ['Adley Rutschman', 'Vinnie Pasquantino', 'Max Muncy', 'Xander Bogaerts', 'Jake Burger', 'Ryan McMahon', 'Alec Bohm', 'Kyle Tucker', "Tyler O'Neill", 'Alex Verdugo', 'Andrew Benintendi', 'Ji-Hwan Bae', 'Jose Altuve', 'Josh Naylor']


In [7]:
h = optimize_team('Harvey')

Harvey
12096
12096
2.689226586203938 ('Dustin May', 'Tony Gonsolin', 'Nestor Cortes', 'Edward Cabrera', 'Jon Gray', 'Kyle Wright', 'George Kirby', 'Anthony DeSclafani', 'Alex Lange')
10.897993058261724 ['William Contreras', 'Jose Abreu', 'Jonathan India', 'Anthony Volpe', 'Rafael Devers', 'Amed Rosario', 'Brian Anderson', 'Fernando Tatis', 'Jazz Chisholm', 'Kyle Schwarber', 'Corbin Carroll', 'James Outman', 'Julio Rodriguez', 'Patrick Wisdom']


In [7]:
w = optimize_team('Wu Tang')

Wu Tang
2856
2856
10.277189317526803 ('Sandy Alcantara', 'Spencer Strider', 'Shane Bieber', 'Bryce Elder', 'Blake Snell', 'Chris Bassitt', 'Sean Manaea', 'Michael King', 'Adam Ottavino')
-5.417515519971084 ['Cal Raleigh', 'Nate Lowe', 'Nico Hoerner', 'Bryson Stott', 'Manny Machado', 'Kolten Wong', 'Wil Myers', 'Mike Trout', 'Luis Robert', 'Cody Bellinger', 'Masataka Yoshida', 'Brent Rooker', 'Ronald Acuna', 'Ramon Laureano']


In [8]:
u = optimize_team('Ugly Spuds')

Ugly Spuds
84672
84672
0.7489042405110831 ('Yu Darvish', 'Framber Valdez', 'Luis Castillo', 'Drew Rasmussen', 'Brandon Woodruff', 'Tyler Mahle', 'Giovanny Gallegos', 'Jorge Lopez', 'James Karinchak')
-6.706208028133138 ['Will Smith', 'C.J. Cron', 'Isaac Paredes', 'Bo Bichette', 'Alex Bregman', 'Corey Seager', 'Josh Jung', 'Nick Castellanos', 'Mike Yastrzemski', 'Christian Yelich', 'Jack Suwinski', 'Oscar Gonzalez', 'Harrison Bader', 'Harold Ramirez']


In [11]:
df.loc[df['Player'].isin(['Ryan McMahon', "Tyler O'Neill"]), 'Owner'] = 'Wu Tang'
df.loc[df['Player'].isin(['Sandy Alcantara', 'Andrew Chafin']), 'Owner'] = 'Lima Time'

In [12]:
l2 = optimize_team('Lima Time')
print('\n')
w2 = optimize_team('Wu Tang')

Lima Time
3360
3360
12.310923108959017 ('Justin Verlander', 'Shane McClanahan', 'Cristian Javier', 'Lucas Giolito', 'Charlie Morton', 'Justin Steele', 'Erik Swanson', 'Jason Adam', 'Sandy Alcantara')
-13.945197960182128 ['Adley Rutschman', 'Josh Naylor', 'Max Muncy', 'Xander Bogaerts', 'Jake Burger', 'Ji-Hwan Bae', 'Vinnie Pasquantino', 'Kyle Tucker', 'Alex Verdugo', 'Andrew Benintendi', 'TJ Friedl', 'Trevor Larnach', 'Jose Altuve', 'Alec Bohm']


Wu Tang
104160
104160
2.7421537317212508 ('Spencer Strider', 'Shane Bieber', 'Bryce Elder', 'Blake Snell', 'Chris Bassitt', 'Sean Manaea', 'Hunter Greene', 'Michael King', 'Adam Ottavino')
3.162005395564864 ['Cal Raleigh', 'Nate Lowe', 'Nico Hoerner', 'Bryson Stott', 'Ryan McMahon', 'Kolten Wong', 'Manny Machado', "Tyler O'Neill", 'Mike Trout', 'Ronald Acuna', 'Ramon Laureano', 'Cody Bellinger', 'Luis Robert', 'Masataka Yoshida']


In [16]:
df = load()
df.loc[df['Player'].isin(['Alex Bregman', 'Yu Darvish']), 'Owner'] = 'Lima Time'
df.loc[df['Player'].isin(['Alec Bohm', 'Justin Verlander']), 'Owner'] = 'Ugly Spuds'

In [17]:
l2 = optimize_team('Lima Time')
print('\n')
w2 = optimize_team('Ugly Spuds')

Lima Time
124956
124956
4.403550267253626 ('Shane McClanahan', 'Cristian Javier', 'Lucas Giolito', 'Charlie Morton', 'Justin Steele', 'Jordan Montgomery', 'Erik Swanson', 'Jason Adam', 'Yu Darvish')
-1.5287369643198963 ['Adley Rutschman', 'Vinnie Pasquantino', 'Max Muncy', 'Xander Bogaerts', 'Jake Burger', 'Ryan McMahon', 'Alex Bregman', 'Kyle Tucker', "Tyler O'Neill", 'Alex Verdugo', 'Andrew Benintendi', 'Ji-Hwan Bae', 'Jose Altuve', 'Josh Naylor']


Ugly Spuds
141120
141120
1.8660023719271694 ('Justin Verlander', 'Framber Valdez', 'Luis Castillo', 'Drew Rasmussen', 'Brandon Woodruff', 'Tyler Mahle', 'Giovanny Gallegos', 'Jorge Lopez', 'James Karinchak')
-9.581041050785146 ['Will Smith', 'Alec Bohm', 'Isaac Paredes', 'Corey Seager', 'Josh Jung', 'Bo Bichette', 'C.J. Cron', 'Nick Castellanos', 'Mike Yastrzemski', 'Christian Yelich', 'Jack Suwinski', 'Oscar Gonzalez', 'Harrison Bader', 'Harold Ramirez']


In [15]:
l3 = optimize_team('Lima Time')
print('\n')
w3 = optimize_team('Ugly Spuds')

Lima Time
2520
2520
11.193824977542931 ('Shane McClanahan', 'Cristian Javier', 'Lucas Giolito', 'Charlie Morton', 'Justin Steele', 'Erik Swanson', 'Jason Adam', 'Yu Darvish', 'Sandy Alcantara')
-11.070364937530119 ['Adley Rutschman', 'Josh Naylor', 'Max Muncy', 'Xander Bogaerts', 'Jake Burger', 'Ji-Hwan Bae', 'Vinnie Pasquantino', 'Kyle Tucker', 'Alex Verdugo', 'Andrew Benintendi', 'TJ Friedl', 'Trevor Larnach', 'Jose Altuve', 'Alex Bregman']


Ugly Spuds
141120
141120
1.8660023719271694 ('Justin Verlander', 'Framber Valdez', 'Luis Castillo', 'Drew Rasmussen', 'Brandon Woodruff', 'Tyler Mahle', 'Giovanny Gallegos', 'Jorge Lopez', 'James Karinchak')
-9.581041050785146 ['Will Smith', 'Alec Bohm', 'Isaac Paredes', 'Corey Seager', 'Josh Jung', 'Bo Bichette', 'C.J. Cron', 'Nick Castellanos', 'Mike Yastrzemski', 'Christian Yelich', 'Jack Suwinski', 'Oscar Gonzalez', 'Harrison Bader', 'Harold Ramirez']


In [14]:
df = load()
df.loc[df['Player'].isin(['Grayson Rodriguez']), 'Owner'] = 'Harvey'
df.loc[df['Player'].isin(['James Outman']), 'Owner'] = 'Lima Time'

In [15]:
l4 = optimize_team('Lima Time')
print('\n')
h4 = optimize_team('Harvey')

Lima Time
598192
598192
5.520648398669712 ('Justin Verlander', 'Shane McClanahan', 'Cristian Javier', 'Lucas Giolito', 'Charlie Morton', 'Justin Steele', 'Jordan Montgomery', 'Erik Swanson', 'Jason Adam')
-2.8852745859699453 ['Adley Rutschman', 'Vinnie Pasquantino', 'Max Muncy', 'Xander Bogaerts', 'Ryan McMahon', 'Ji-Hwan Bae', 'Alec Bohm', 'James Outman', 'Kyle Tucker', "Tyler O'Neill", 'Alex Verdugo', 'Andrew Benintendi', 'Jose Altuve', 'Josh Naylor']


Harvey
2328
2328
2.689226586203938 ('Dustin May', 'Tony Gonsolin', 'Nestor Cortes', 'Edward Cabrera', 'Jon Gray', 'Kyle Wright', 'George Kirby', 'Anthony DeSclafani', 'Alex Lange')
6.998851462626627 ['William Contreras', 'Jose Abreu', 'Jonathan India', 'Anthony Volpe', 'Rafael Devers', 'Amed Rosario', 'Triston Casas', 'Fernando Tatis', 'Jazz Chisholm', 'Kyle Schwarber', 'Corbin Carroll', 'Brian Anderson', 'Julio Rodriguez', 'Patrick Wisdom']


In [ ]:
df[['Player', 'BA', 'HR', 'SB', 'R', 'RBI', 'z']].sort_values('z', ascending=False)

In [91]:
df[df['Player'].isin(['Justin Verlander', 'Alec Bohm', 'Alex Bregman', 'Yu Darvish'])] .groupby(['Owner', 'Player'])[['HR', 'SB', 'R', 'RBI', 'BA', 'z', 'IP', 'ERA', 
                                                                         'WHIP', 'W', 'SO', 'Sv+Hld']].sum()

HR   SB     R   RBI     BA         z     IP  \
Owner      Player                                                            
Lima Time  Alex Bregman      19.0  1.0  78.0  73.0  0.271  0.694379    0.0   
           Yu Darvish         0.0  0.0   0.0   0.0  0.000  2.272376  157.0   
Ugly Spuds Alec Bohm         11.0  2.0  52.0  67.0  0.282 -2.180454    0.0   
           Justin Verlander   0.0  0.0   0.0   0.0  0.000  3.389474  151.0   

                              ERA  WHIP     W     SO  Sv+Hld  
Owner      Player                                             
Lima Time  Alex Bregman      0.00  0.00   0.0    0.0     0.0  
           Yu Darvish        3.21  1.11  11.0  163.0     0.0  
Ugly Spuds Alec Bohm         0.00  0.00   0.0    0.0     0.0  
           Justin Verlander  2.80  1.16  14.0  156.0     0.0

In [ ]:
df.loc[df['Player']=='Grayson Rodriguez', ['W', 'SO', 'ERA', 'WHIP']] = [7, 131, 3.8, 1.2]

In [ ]:
df[df['Player'].isin(w.pitcher_optimized_lineup+l.pitcher_optimized_lineup)].groupby('Owner').apply(lambda s: pd.Series({
    'W': s['W'].sum(),
    'SO':s['SO'].sum(),
    'S':s['S'].sum()+s['HD'].sum(),
    'ERA':round(s['ER'].sum()/s['IP'].sum()*9,2),
    'WHIP':round((s['Ha'].sum()+s['BBa'].sum())/s['IP'].sum(),2),
}))

In [ ]:
df[df['Player'].isin(['Jose Abreu', 'Vinnie Pasquantino', 'Justin Steele', 'Brian Anderson', 'Amed Rosario', 
                      'Reynaldo Lopez'])] .groupby(['Owner', 'Player'])[['AB','HR', 'SB', 'R', 'RBI', 'BA', 'z', 'IP', 'ERA', 
                                                                         'WHIP', 'W', 'SO', 'Sv+Hld']].sum()

In [ ]:
df[df['Player'].isin(['Jose Abreu', 'Vinnie Pasquantino', 'Justin Steele', 'Brian Anderson', 'Amed Rosario', 
                      'Reynaldo Lopez'])] .groupby('Owner')[['AB','HR', 'SB', 'R', 'RBI', 'BA', 'z', 'IP', 'ERA', 
                                                                         'WHIP', 'W', 'SO', 'Sv+Hld']].sum()

In [ ]:
h.pitcher_optimized_lineup, l.pitcher_optimized_lineup

In [ ]:
print(f"Harvey\nhitting: {h.hitter_optimized_z}\npitching: {h.pitcher_optimized_z}")
#print(f"Lima\nhitting: {l.hitter_optimized_z}\npitching: {l.pitcher_optimized_z}")

In [75]:
ol_dict = {}
for t in df.Owner.sort_values().unique():
    print(t)
    ol_dict[t] = ol.Optimized_Lineups(t, df[df['Owner']==t])
    ol_dict[t]._make_pitcher_combos()
    ol_dict[t]._make_hitter_combos()
    print(f'Offense: {ol_dict[t].hitter_optimized_z}')
    print(f'Pitching: {ol_dict[t].pitcher_optimized_z}')
    print(f'Total z: {ol_dict[t].pitcher_optimized_z + ol_dict[t].hitter_optimized_z}')

Brewbirds
21060
21060
Offense: 0.31308555284945294
Pitching: 1.9621198543214435
Total z: 2.2752054071708967
Charmer
66000
66000
Offense: -6.1254861692331515
Pitching: 7.284372059363754
Total z: 1.1588858901306027
Dirty Birds
72720
72720
Offense: 3.7647123877350834
Pitching: -0.5255299489434608
Total z: 3.2391824387916226
Harvey
12096
12096
Offense: 10.897993058261724
Pitching: 2.689226586203938
Total z: 13.587219644465662
Lil Trump
0
0


ValueError: attempt to get argmax of an empty sequence

In [ ]:
for k, v in ol_dict['Brewbirds'].h_dict.items():
    print(k,v['all_pos'])
    if "OF" in v['all_pos']:#.replace('[','').replace(']','').replace('\'','').split(", "):
        print('yes')

In [ ]:
def get_team_projections(owner):
    b = pd.read_html(str(soup.find_all('table')[1]), header=1, converters={'AB':str_2_int,'H':str_2_int, 'R':str_2_int,'HR':str_2_int,'RBI':str_2_int,'SB':str_2_int})[0].drop(columns='Edit')
    b = b[b['Pos']!='Reserves'].reset_index(drop=False).drop(columns='index')
    b.loc[b.index<14, 'Status'] = 'Start'
    b.loc[b.index>=14, 'Status'] = 'Bench'
    b['Name'] = b.Players.apply(lambda x: parse_name(x))
    b['Elig'] = b.Players.apply(lambda x: parse_pos(x))
    b['Team'] = b.Players.apply(lambda x: parse_team(x))
    b['Owner'] = owner
    
    p = pd.read_html(str(soup.find_all('table')[2]), header=1, converters={'INNs':str_2_int,'BBI':str_2_int, 'HA':str_2_int, 'W':str_2_int,'S':str_2_int,'K':str_2_int})[0].drop(columns='Edit')
    p = p[~p['Pos'].str.contains('Reserve')].reset_index(drop=False).drop(columns='index')
    p['ER'] = (p['ERA'].astype(float)/9)*p['INNs'].astype(float)
    p.loc[p.index<9, 'Status'] = 'Start'
    p.loc[p.index>=9, 'Status'] = 'Bench'
    p['Name'] = p.Players.apply(lambda x: parse_name(x))
    p['Elig'] = p.Players.apply(lambda x: parse_pos(x))
    p['Team'] = p.Players.apply(lambda x: parse_team(x))
    p['Owner'] = owner
    return b, p

In [ ]:
#df = b[b['Status']=='Start'].groupby('Owner').agg({'R':'sum', 'RBI':'sum', 'HR':'sum', 'SB':'sum', 'H':'sum', 'AB':'sum'})
def get_proj_batting_stats_by_owner(b):
    df = b[b['Status']=='Start'].groupby("Owner").apply(lambda s: pd.Series({ 
        "HR": s["HR"].sum(), 
        "R": s["R"].sum(),
        'RBI': s['RBI'].sum(),
        'SB': s['SB'].sum(),
        "BA": s["H"].sum()/s["AB"].sum(),
        'H': s['H'].sum(),
        'AB': s['AB'].sum()
    }))
    return df

def get_batting_ranks(proj):
    b_ranks = proj.drop(columns=['H', 'AB']).rank()
    b_ranks['Pts'] = b_ranks.sum(axis=1)
    return b_ranks

proj_batting_stats_by_owner = get_proj_batting_stats_by_owner(b)
b_ranks = get_batting_ranks(proj_batting_stats_by_owner)

In [ ]:
dfp = p[p['Status']=='Start'].groupby("Owner").apply(lambda s: pd.Series({ 
    "W": s["W"].sum(), 
    "K": s["K"].sum(),
    'S': s['S'].sum(),
    'ERA': s['ER'].sum()/s['INNs'].sum()*9,
    "WHIP": (s["HA"].sum()+s['BBI'].sum())/s["INNs"].sum(),
}))

p_ranks = dfp.rank()
p_ranks['Pts'] = p_ranks.sum(axis=1)
p_ranks.sort_values('Pts', ascending=False)

In [ ]:
dfp.sort_values('W', ascending=False)

In [ ]:
(b_ranks['Pts']+p_ranks['Pts']).sort_values().plot(kind='bar')

In [ ]:
z_table = b[b['Status']=='Start'].agg({'HR':['mean','std'], 'R':['mean', 'std'], 'RBI':['mean','std'], 'SB':['mean','std'], 'H':['mean','std'], 'AB':['mean','std']})
z_table = z_table.join(p[p['Status']=='Start'].agg({'W':['mean','std'], 'K':['mean', 'std'], 'S':['mean','std'], 'ER':['mean','std'], 'HA':['mean','std'], 'BBI':['mean','std'], 'INNs':['mean', 'std']}))
z_table

In [ ]:
for stat in ['R', 'HR', 'RBI', 'SB']:
    b['z'+stat] = (b[stat]-z_table[stat]['mean']) / z_table[stat]['std']

b['zBA'] = b['H']-(b['AB']*(z_table['H']['mean']) / z_table['AB']['mean'])
b['zBA'] = (b['zBA']-b['zBA'].mean())/b['zBA'].std()
b['z'] = b['zBA'] + b['zHR'] + b['zR'] + b['zRBI'] + b['zSB']

for stat in ['W', 'K', 'S']:
    p['z'+stat] = (p[stat]-z_table[stat]['mean']) / z_table[stat]['std']

p['zER'] = ((p['ER']*9) - ((p['INNs'] * z_table['ER']['mean'] * 9) / z_table['INNs']['mean'])) * -1
p['zERA'] = (p['zER'] - p['zER'].mean()) / p['zER'].std()
p['zH+BB'] = ((p['HA']+p['BBI']) - (p['INNs'] * ((z_table['HA']['mean'] + z_table['BBI']['mean']) / z_table['INNs']['mean']))) * -1
p['zWHIP'] = (p['zH+BB'] - p['zH+BB'].mean()) / p['zH+BB'].std()
p['z'] = p['zERA'] + p['zWHIP'] + p['zW'] + p['zS'] + p['zK']

In [ ]:
p['zER'].mean(), p['zER'].std()
p.sort_values('z').tail(15)

In [ ]:
conv = (260*12)/(b[b['z']>0].z.sum() + p[p['z']>0].z.sum())
conv

In [ ]:
b['$'] = b['z'] * conv *.65
p['$'] = p['z'] * conv *.35

In [ ]:
b[b['Status']=='Start'].groupby('Owner')['$'].sum()

In [ ]:
#ba_pts = row['H']-(row['AB']*(qual_avgs['H'][0]/qual_avgs['AB'][0]))
#zBA = (ba_pts-qual_avgs['zlgBA'][0])/qual_avgs['zlgBA'][1]
play = b[(b['Owner']=='Lima Time') & (b['Status']=='Start')]
bench = b[(b['Owner']=='Lima Time') & (b['Status']=='Bench')]

In [ ]:
play

In [ ]:
bench

In [ ]:
def get_position_list(player):
    elig = player.split(',')
    for pos in elig:
        if pos in ['1B', '3B']:
            elig.extend(['CI'])
        elif pos in ['2B', 'SS']:
            elig.extend(['MI'])
    elig.extend(['DH'])
    return list(dict.fromkeys(elig))

def check_if_better(cur, new):
    if new > cur:
        return new
    else:
        return cur

In [ ]:
# Line-up Optimization Algorithm
base_z = play['z'].sum()

for bench_spot in range(bench.shape[0]):
    optimal_z = play['z'].sum()

    for i in get_position_list(bench['Elig'].iloc[bench_spot]):
        mask = (play['Pos']==i)
        if i in ['C', '1B', '2B', 'SS', '3B', 'MI', 'CI']:
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[0], optimal_z)
        if i=='OF':
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[0], optimal_z)
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[1], optimal_z)
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[2], optimal_z)
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[3], optimal_z)
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[4], optimal_z)
        if i=='DH':
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[0], optimal_z)
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[1], optimal_z)
    
    if optimal_z > play['z'].sum():
        print('The line-up is not optimal.',bench.iloc[bench_spot]['Name'],'should go in.')
    else:
        print('The line-up is already optimal.',bench.iloc[bench_spot]['Name'],'should stay on the bench.')

In [ ]:
b[b['Elig'].str.contains('OF')].groupby('Owner')['z'].mean()

In [ ]:
b[b['Owner']=='Brewbirds']

In [ ]:
trade = b.copy()

In [ ]:
trade.loc[trade['Name']=='Nolan Arenado', 'Owner'] = 'Lima Time'
trade.loc[trade['Name']=='Tyler O\'Neill', 'Owner'] = 'Dirty Birds'

In [ ]:
trade[trade['Name'].isin(['Ryan McMahon', 'Tyler O\'Neill'])]

In [ ]:
get_proj_batting_stats_by_owner(trade)

In [ ]:
proj_batting_stats_by_owner

In [ ]:
get_batting_ranks(get_proj_batting_stats_by_owner(trade)).sort_values('Pts', ascending=False)

In [ ]:
from pybaseball.analysis.projections.marcels import MarcelProjectionsPitching
from pybaseball.analysis.projections.marcels import MarcelProjectionsBatting
from pybaseball.lahman import people

In [ ]:
ppl = people()  # for merging player names onto the projections
ppl['Player'] = ppl['nameFirst'].str.cat(ppl['nameLast'], ' ')
marcel_batting = MarcelProjectionsBatting()
marcel_pitching = MarcelProjectionsPitching()

In [ ]:
pitchers = marcel_pitching.projections(2022)
batters = marcel_batting.projections(2022)
pitchers = pd.merge(ppl[['playerID', 'Player']], pitchers, on='playerID', how='right')
batters = pd.merge(ppl[['playerID', 'Player']], batters, on='playerID', how='right')

In [ ]:
pitchers.sort_values('SO', ascending=False).head()

In [ ]:
batters.sort_values('HR', ascending=False).head(20)